In [21]:
import struct

with open("source_test.bin","wb") as fd:
    bin_write=struct.pack("B",97)
    fd.write(bin_write)
    fd.write(bin_write)
    fd.write(bin_write)
    fd.write(bin_write)
    


for i=1;                    p=n*R1/K    
for i=2,...,(k/R2-1);       p=n/(i(i-1)(i-iR2/K))   
for i=K/R2,....K;           p=n*HW(i)   

这里可以把density看作度分布使用的rom里面存的东西，精度就是位数。

但是度如何选择已经知道了。接下来是如何uniform choice the，里面存的是一个数字。

叫做associator

### LT最重要的就是随机数生成算法。

如果生成矩阵A，在这里只需要关心k。

### 关于esi的产生。
1. 我觉得可以暂时先不考虑，因为无非就是两种，一种是压缩产生的信息矩阵，然后接受端只需要解压缩就可以。
可是如果k很大，就会很奇怪。
2. 另一种做法，就是利用key，作为随机数，然后产生一些随机数的序列信息，然后接受端重现随机数产生，并选择相应的模块。这样的做法是减少传输开销，但是会增加复杂度。
### 关于随机数的产生
没有仔细的研究，基本上都是使用lfsr这个方法，只是修正lfsr支持2××n。

然后关于度选择，之后的associator，利用（nx+y）%prime（k）的方法

说明：
    1. n 就是随机数序列
    2. x随机数range（1，prime（k））
    3. y随机数，range(0,print(k))
    4. primt（k）大于等于k的最大 ，range（k，prime（k）），如果选中就剔除
    5. 相当于一个需要 多个周期才能产生一个

关于度选择 ，简单来说可以分为两种。第一种就是利用比较器加rom，另一种就是纯用rom。
简单倒是很简单。。两个我也都会实现。
### 写verilog。理论上来说这个度选择核associator，是要能够生成python代码，和相应的比较mif文件的

In [1]:
### 构建鲁棒弧度子分布，还有想一下度分布生成。
from math import *
class RSD:
    def __init__(self,k,knpPrecision=20):
        '''里面使用到的参数，都是根据luby的一个专利文档来de 
        该文档仅仅只有一个参数，剩下的参数，基本上都是来源于经验。
        '''
        ##load the parameter 
        

        self.k=k
        ## the precision bit 
        self.knpPrecision=knpPrecision
    def patent_rsd(self,):
        self.__parameter_init()
        ##R1:start_ripple size 
        self.R1=int(self.kdSFactor*sqrt(self.k))
        ##R2:ripple target size
        self.R2=int(self.knRAdd+self.kdRFactor*k**0.25)
        ### 可能出现不了3中情况，因为R1和R2的选择。
        r_kinds=int(self.k/self.R2)-2
        if r_kinds<1:
            r_kinds=1
            raise Exception("not good")
        elif r_kinds>self.k:
            r_kinds=K
            raise Exception("not good")
        
        tail_kinds=1
        tail=self.R2
        while(tail>1):
            tail=tail//2
            tail_kinds+=1

        kinds=tail_kinds+r_kinds
        if kinds>k:
            tail_kinds=k-r_kinds
            kinds=k
        
        ##############
        kind_weight=[0]*kinds
        kind_density=[0]*kinds
        kind_frac=[0]*kinds
        ##weight 1
        kind_weight[0]=1
        kind_frac[0]=self.R1/self.k
        ##
        for i in range(1,r_kinds):
            kind_frac[i]=kind_frac[i-1]+1/(i*(i+1)*(1-(i+1)*self.R2/self.k))
            kind_weight[i]=i+1
        n_power=1
        for i in range(tail_kinds):
            n_power=n_power*2
        print(n_power>self.R2)
        j=2
        for i in range(r_kinds,kinds):
            kind_frac[i]=kind_frac[i-1]+self.fdTFactor*n_power/(j*k)
            kind_weight[i]=int(j*k/n_power)
            j=j*2

        ###精确度问题20位精确度
        modulus=1<<self.knpPrecision
        for i in range(kinds):
            kind_density[i]=int(kind_frac[i]/kind_frac[-1]*modulus)
        ######最大重量
        self.max_weight=max(kind_weight)
        print(kind_weight)
        #### 计算平均
        sum=kind_weight[0]*kind_density[0]
        for i in range(kinds):
            sum=sum+int(kind_weight[i]*(kind_density[i]-kind_density[i-1]))
        self.average_weight=int(sum/modulus+1)
        
        self.kind_density=kind_density
        self.kind_weight=kind_weight
    def __parameter_init(self):
        '''
        从那个文章里面作者提供的参数,
        '''
        self.knRAdd=2
        self.kdRFactor=2.0

        self.kdSFactor=1.4

        self.fdTFactor=1.6
        
    def lt_rsd(self,):
        '''
        在lt_isd的基础上叠加一个，修正版本，然后再归一化。
        
        '''
        pass
    def lt_isd(self,):
        pass
        k=self.k
        kinds=self.k
        kind_weight=[0]*kinds
        kind_frac=[0]*kinds
        kind_density=[0]*kinds
        modulus=1<<self.knpPrecision
        ### 定义概率
        for i in range(k):
            kind_weight[i]=i+1
            if i==0:
                kind_frac[i]=1/self.k
            else:
                kind_frac[i]=kind_frac[i-1]+1/(i*(i+1))
            
        modulus=1<<self.knpPrecision
        for i in range(kinds):
            kind_density[i]=int(kind_frac[i]/kind_frac[-1]*modulus)
        ######最大重量
        self.max_weight=max(kind_weight)
        print(kind_weight)
        #### 计算平均
        sum=kind_weight[0]*kind_density[0]
        for i in range(kinds):
            sum=sum+int(kind_weight[i]*(kind_density[i]-kind_density[i-1]))
        self.average_weight=int(sum/modulus+1)
        
        self.kind_density=kind_density
        self.kind_weight=kind_weight
        

        
    def generate_random_v1(self):
        '''
        generate verilog ,mif,还有编码矩阵A的二进制文件。
        '''                
        pass
    


In [2]:
aa=RSD(30)

In [3]:
aa.lt_isd()


ZeroDivisionError: division by zero

In [8]:
hex(148)

'0x94'